In [1]:
import yaml
from modules.agent import OpenAICommentAnalysisAgent


with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')


In [ ]:
agent.comment_analyze('This desk is sturdy!')

In [ ]:
resp = agent.points_extract("I spent up on the desktop but the slab of wood was beautiful (and heavy). The legs and other materials appear to be all metal. Wood + metal is rare these days everything is plastic.   Have only owned it for a week but love it so far. Plan to keep for the rest of my life.")
print(resp)

In [ ]:
import pandas as pd


df = pd.read_csv('./docs/testset_amz_bsr_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

# Scratchs

In [ ]:
import re
import yaml
import pandas as pd
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent



def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches


df = pd.read_csv('./docs/testset_stations_bs_reviews.csv', sep='\t')
print(df.columns)
print(df.shape)

with open('openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations', model='gpt-4o-2024-08-06')

for i in tqdm(range(len(df))):
    comment = df.iloc[i]['content']
    if not isinstance(comment, str):
        continue

    points = agent.points_extract(comment)
    # print(points)
    df.loc[i, '体验点分析'] = points
    extract_points = extract_output(points)
    df.loc[i, '体验点'] = ''.join(extract_points)
    # print(extract_points[0])
    # break

df.to_csv('./results/stations_bs_reviews_points.csv', sep='\t')

In [1]:
import re
import yaml
import pandas as pd
from tqdm import tqdm
from modules.agent import OpenAICommentAnalysisAgent

with open('./openai_keys.yaml', 'r') as f:
    data = yaml.safe_load(f)

agent = OpenAICommentAnalysisAgent(openai_key=data['openai']['key'], criteria='stations')


def extract_output(text):
    pattern = r'\{.*?\}'
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches

df = pd.read_csv('./results/testset_stations.csv', sep='\t')
df = df.drop(columns=[col for col in df.columns if "Unname" in col])
df['反审查过程'] = ''
print(df.columns)

for i in tqdm(range(len(df))):
    comment = df.iloc[i]['英文评论']
    pre_points = df.iloc[i]['分析结果']
    
    check_result = agent.inspect(comment, pre_points)


    clean_r = extract_output(check_result)[-1]
    df.loc[i, '反审查过程'] = check_result
    df.loc[i, '反审查结果'] = str(clean_r)

df.to_csv('./results/testset_stations.csv', sep='\t', index=False)

Index(['title', 'content', '内容', '英文评论', '英文分析', '分析结果', '反审查结果', '人审(Tom)',
       '备注(Tom)', '反审查过程'],
      dtype='object')


100%|██████████| 30/30 [02:30<00:00,  5.03s/it]
